In [1]:
#pip install category_encoders

In [2]:
#pip install catboost

In [3]:
#pip install ipywidgets

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import cycle, islice
import datetime as datetime
import glob
from matplotlib import pyplot
import plotly.graph_objects as go
import plotly.io as pio
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss, make_scorer
import category_encoders as ce
from sklearn.ensemble import RandomForestRegressor
import warnings

%matplotlib inline

plt.style.use('default')
sns.set(style="whitegrid")
pd.options.display.float_format = '{:20,.2f}'.format
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (15, 5)


import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree
from catboost import CatBoostRegressor

# Train

__Lectura de datos__

In [5]:
train = pd.read_csv('Train_TP2_Datos_2020-2C.csv')

train = train[(train.Stage == 'Closed Won') | (train.Stage == 'Closed Lost')]
train['Stage'] = train['Stage'].transform(func=lambda x : 1 if x=='Closed Won' else 0)

train = train.drop('Last_Activity', axis=1) #Todos datos nulos
train = train.drop('Actual_Delivery_Date', axis=1) #Todos datos nulos
train = train.drop('Submitted_for_Approval', axis=1) #Todos los datos en cero
train = train.drop('Prod_Category_A', axis=1) #Todos en none
train = train.drop('Sales_Contract_No', axis=1) #Para evitar el problema de target leakage
train = train.drop('Opportunity_Name', axis=1) #Redundante si tenemos Opportunity_ID

***

__FEATURES__

__Aprobación especial de su precio total__

In [6]:
train['Pricing, Delivery_Terms_Quote_Appr'] = train['Pricing, Delivery_Terms_Quote_Appr']\
                                                .transform(func=lambda x : 'Necesita' if x==1 else 'NoNecesita')
train['Pricing, Delivery_Terms_Approved'] = train['Pricing, Delivery_Terms_Approved']\
                                                .transform(func=lambda x : 'Obtuvo' if x==1 else 'NoObtuvo')

train['Aprobacion_Especial'] = train['Pricing, Delivery_Terms_Quote_Appr']+train['Pricing, Delivery_Terms_Approved']

train = train.drop('Pricing, Delivery_Terms_Quote_Appr', axis=1)
train = train.drop('Pricing, Delivery_Terms_Approved', axis=1)

***

__TRF promedio__

In [7]:
train['TRF_promedio'] = train.groupby('Opportunity_ID')['TRF'].transform('mean')
train = train.drop('TRF', axis=1)

***

__Total Taxable Amount (converted)__

In [8]:
#Completo datos faltantes con Total Amount
train['Total_Taxable_Amount'].loc[train.Total_Taxable_Amount==0.00] = train.loc[train.Total_Taxable_Amount==0.00]\
                                            .groupby(['Opportunity_ID'])['Total_Amount'].transform('sum')

#Convierto a una misma moneda (a dólares) con la información que ya viene en ASP, ASP_Currency y ASP_(converted)
toEUR = train[train.ASP_Currency=='EUR'].head(1)[['ASP_(converted)']].iloc[0,0]/train[train.ASP_Currency=='EUR']\
        .head(1)[['ASP']].iloc[0,0]
toAUD = train[train.ASP_Currency=='AUD'].head(1)[['ASP_(converted)']].iloc[0,0]/train[train.ASP_Currency=='AUD']\
        .head(1)[['ASP']].iloc[0,0]
toGBP = train[train.ASP_Currency=='GBP'].head(1)[['ASP_(converted)']].iloc[0,0]/train[train.ASP_Currency=='GBP']\
        .head(1)[['ASP']].iloc[0,0]
toJPY = train[['ASP_(converted)']].iloc[12,0]/train[['ASP']].iloc[12,0]
#En el head(1) de JPY justo habia un nan. Ese numero magico 12 corresponde a la primera fila de moneda JPY con un valor valido

train['Total_Taxable_Amount_(converted)']=0.0
train['Total_Taxable_Amount_(converted)'].loc[train['Total_Taxable_Amount_Currency']=='EUR'] = \
                                                        train['Total_Taxable_Amount'].apply(lambda x: x*toEUR)
train['Total_Taxable_Amount_(converted)'].loc[train['Total_Taxable_Amount_Currency']=='JPY'] = \
                                                        train['Total_Taxable_Amount'].apply(lambda x: x*toJPY)
train['Total_Taxable_Amount_(converted)'].loc[train['Total_Taxable_Amount_Currency']=='AUD'] = \
                                                        train['Total_Taxable_Amount'].apply(lambda x: x*toAUD)
train['Total_Taxable_Amount_(converted)'].loc[train['Total_Taxable_Amount_Currency']=='GBP'] = \
                                                        train['Total_Taxable_Amount'].apply(lambda x: x*toGBP)
train['Total_Taxable_Amount_(converted)'].loc[train['Total_Taxable_Amount_Currency']=='USD'] = \
                                                        train['Total_Taxable_Amount'].apply(lambda x: x)

train = train.drop('Total_Taxable_Amount_Currency', axis=1)
train = train.drop('Total_Amount_Currency', axis=1)
train = train.drop('Total_Amount', axis=1)
train = train.drop('ASP_Currency', axis=1)
train = train.drop('ASP', axis=1)
train = train.drop('ASP_(converted)_Currency', axis=1)
train = train.drop('ASP_(converted)', axis=1)

***

__Tiempo de entrega (en días)__

In [9]:
#Es la diferencia entre la maxima fecha de entrega (Planned_Delivery_End_Date) y la mínima (Planned_Delivery_Start_Date)
#Reemplazo los Nat de Planned_Delivery_End_Date con los de Start_Date (curiosamente da mejor resultado que
#reemplazarlo con la fecha de Month)
train['Planned_Delivery_End_Date'].loc[train['Planned_Delivery_End_Date']=='NaT'] = train['Planned_Delivery_Start_Date']

train['Planned_Delivery_Start_Date'] = pd.to_datetime(train['Planned_Delivery_Start_Date'])
train['Planned_Delivery_End_Date'] = pd.to_datetime(train['Planned_Delivery_End_Date'])

train['Planned_Delivery_Start_Date_min'] = train.groupby('Opportunity_ID')['Planned_Delivery_Start_Date'].transform('min')
train['Planned_Delivery_End_Date_max'] = train.groupby('Opportunity_ID')['Planned_Delivery_End_Date'].transform('max')

train['Diferencia'] = train['Planned_Delivery_End_Date_max'] - train['Planned_Delivery_Start_Date_min']
train['Diferencia'] = train['Diferencia'].dt.days

train = train.drop('Planned_Delivery_Start_Date', axis=1)
train = train.drop('Planned_Delivery_End_Date', axis=1)
train = train.drop('Planned_Delivery_Start_Date_min', axis=1)
train = train.drop('Planned_Delivery_End_Date_max', axis=1)

***

__Cantidad de productos por oportunidad__

In [10]:
train['Cantidad de productos'] = train.groupby('Opportunity_ID')['ID'].transform('count')
train = train.drop('ID', axis=1)

***

__Elimino lo que ya no voy a usar__

In [11]:
train = train.drop('Bureaucratic_Code_0_Approval', axis=1)
train = train.drop('Bureaucratic_Code_0_Approved', axis=1)
train = train.drop('Account_Created_Date', axis=1)
train = train.drop('Source ', axis=1)
train = train.drop('Billing_Country', axis=1)
train = train.drop('Opportunity_Owner', axis=1)
train = train.drop('Account_Type', axis=1)
train = train.drop('Quote_Type', axis=1)
train = train.drop('Delivery_Terms', axis=1)
train = train.drop('Opportunity_Created_Date', axis=1)
train = train.drop('Product_Type', axis=1)
train = train.drop('Size', axis=1)
train = train.drop('Product_Category_B', axis=1)
train = train.drop('Price', axis=1)
train = train.drop('Currency', axis=1)
train = train.drop('Quote_Expiry_Date', axis=1)
train = train.drop('Last_Modified_Date', axis=1)
train = train.drop('Last_Modified_By', axis=1)
train = train.drop('Product_Family', axis=1)
train = train.drop('Product_Name', axis=1)
train = train.drop('Month', axis=1)
train = train.drop('Territory', axis=1)
train = train.drop('Region', axis=1)
train = train.drop('Opportunity_Type', axis=1)
train = train.drop('Account_Owner', axis=1)
train = train.drop('Delivery_Quarter', axis=1)
train = train.drop('Delivery_Year', axis=1)
train = train.drop('Bureaucratic_Code', axis=1)

#Elimino los duplicados para obtener una oportunidad por fila. Nota: Todos los features que siguen corresponden a
#caracteristicas de las oportunidades, no a caracteristicas especificas de c/producto dentro de una oportunidad
train = train.drop_duplicates('Opportunity_ID')
train = train.drop('Opportunity_ID', axis=1)

***

In [12]:
train_label = train['Stage']
train_data = train.iloc[:, train.columns != 'Stage']

***

# Test

__Lectura de datos__

In [13]:
test = pd.read_csv('Test_TP2_Datos_2020-2C.csv')

test = test.drop('Last_Activity', axis=1)
test = test.drop('Actual_Delivery_Date', axis=1)
test = test.drop('Submitted_for_Approval', axis=1)
test = test.drop('Prod_Category_A', axis=1)
test = test.drop('Sales_Contract_No', axis=1)
test = test.drop('Opportunity_Name', axis=1)

***

__FEATURES__

__Aprobación especial de su precio total__

In [14]:
test['Pricing, Delivery_Terms_Quote_Appr'] = test['Pricing, Delivery_Terms_Quote_Appr']\
                                                .transform(func=lambda x : 'Necesita' if x==1 else 'NoNecesita')
test['Pricing, Delivery_Terms_Approved'] = test['Pricing, Delivery_Terms_Approved']\
                                                .transform(func=lambda x : 'Obtuvo' if x==1 else 'NoObtuvo')

test['Aprobacion_Especial'] = test['Pricing, Delivery_Terms_Quote_Appr']+test['Pricing, Delivery_Terms_Approved']

test = test.drop('Pricing, Delivery_Terms_Quote_Appr', axis=1)
test = test.drop('Pricing, Delivery_Terms_Approved', axis=1)

***

__TRF promedio__

In [15]:
test['TRF_promedio'] = test.groupby('Opportunity_ID')['TRF'].transform('mean')
test = test.drop('TRF', axis=1)

***

__Total Taxable Amount (converted)__

In [16]:
test['Total_Taxable_Amount'].loc[test.Total_Taxable_Amount==0.00] = test.loc[test.Total_Taxable_Amount==0.00]\
                                            .groupby(['Opportunity_ID'])['Total_Amount'].transform('sum')

test['Total_Taxable_Amount_(converted)']=0.0
test['Total_Taxable_Amount_(converted)'].loc[test['Total_Taxable_Amount_Currency']=='EUR'] = \
                                                        test['Total_Taxable_Amount'].apply(lambda x: x*toEUR)
test['Total_Taxable_Amount_(converted)'].loc[test['Total_Taxable_Amount_Currency']=='JPY'] = \
                                                        test['Total_Taxable_Amount'].apply(lambda x: x*toJPY)
test['Total_Taxable_Amount_(converted)'].loc[test['Total_Taxable_Amount_Currency']=='AUD'] = \
                                                        test['Total_Taxable_Amount'].apply(lambda x: x*toAUD)
test['Total_Taxable_Amount_(converted)'].loc[test['Total_Taxable_Amount_Currency']=='GBP'] = \
                                                        test['Total_Taxable_Amount'].apply(lambda x: x*toGBP)
test['Total_Taxable_Amount_(converted)'].loc[test['Total_Taxable_Amount_Currency']=='USD'] = \
                                                        test['Total_Taxable_Amount'].apply(lambda x: x)

test = test.drop('Total_Amount_Currency', axis=1)
test = test.drop('Total_Amount', axis=1)
test = test.drop('Total_Taxable_Amount_Currency', axis=1)
test = test.drop('ASP_Currency', axis=1)
test = test.drop('ASP', axis=1)
test = test.drop('ASP_(converted)_Currency', axis=1)
test = test.drop('ASP_(converted)', axis=1)

***

__Tiempo de entrega (en días)__

In [17]:
test['Planned_Delivery_End_Date'].loc[test['Planned_Delivery_End_Date']=='NaT'] = test['Planned_Delivery_Start_Date']

test['Planned_Delivery_Start_Date'] = pd.to_datetime(test['Planned_Delivery_Start_Date'])
test['Planned_Delivery_End_Date'] = pd.to_datetime(test['Planned_Delivery_End_Date'])

test['Planned_Delivery_Start_Date_min'] = test.groupby('Opportunity_ID')['Planned_Delivery_Start_Date'].transform('min')
test['Planned_Delivery_End_Date_max'] = test.groupby('Opportunity_ID')['Planned_Delivery_End_Date'].transform('max')

test['Diferencia'] = test['Planned_Delivery_End_Date_max'] - test['Planned_Delivery_Start_Date_min']
test['Diferencia'] = test['Diferencia'].dt.days

test = test.drop('Planned_Delivery_Start_Date', axis=1)
test = test.drop('Planned_Delivery_End_Date', axis=1)
test = test.drop('Planned_Delivery_Start_Date_min', axis=1)
test = test.drop('Planned_Delivery_End_Date_max', axis=1)

***

__Cantidad de productos por oportunidad__

In [18]:
test['Cantidad de productos'] = test.groupby('Opportunity_ID')['ID'].transform('count')
test = test.drop('ID', axis=1)

***

__Elimino lo que ya no voy a usar__

In [19]:
test = test.drop('Bureaucratic_Code_0_Approval', axis=1)
test = test.drop('Bureaucratic_Code_0_Approved', axis=1)
test = test.drop('Account_Created_Date', axis=1)
test = test.drop('Source ', axis=1)
test = test.drop('Billing_Country', axis=1)
test = test.drop('Opportunity_Owner', axis=1)
test = test.drop('Account_Type', axis=1)
test = test.drop('Quote_Type', axis=1)
test = test.drop('Delivery_Terms', axis=1)
test = test.drop('Opportunity_Created_Date', axis=1)
test = test.drop('Product_Type', axis=1)
test = test.drop('Size', axis=1)
test = test.drop('Product_Category_B', axis=1)
test = test.drop('Price', axis=1)
test = test.drop('Currency', axis=1)
test = test.drop('Quote_Expiry_Date', axis=1)
test = test.drop('Last_Modified_Date', axis=1)
test = test.drop('Last_Modified_By', axis=1)
test = test.drop('Product_Family', axis=1)
test = test.drop('Product_Name', axis=1)
test = test.drop('Month', axis=1)
test = test.drop('Territory', axis=1)
test = test.drop('Region', axis=1)
test = test.drop('Opportunity_Type', axis=1)
test = test.drop('Account_Owner', axis=1)
test = test.drop('Delivery_Quarter', axis=1)
test = test.drop('Delivery_Year', axis=1)
test = test.drop('Bureaucratic_Code', axis=1)

test = test.drop_duplicates('Opportunity_ID')
IDs = test[['Opportunity_ID']].copy()
test = test.drop('Opportunity_ID', axis=1)

***

# Modelos

## Catboost

In [20]:
LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(train_data, train_label, train_size=0.7, random_state=1234)

In [22]:
categorical_features_indices = np.where(train_data.dtypes == np.object)[0]

In [23]:
%%capture
modelCatBoost=CatBoostRegressor(iterations=250, depth=3, learning_rate=0.05)
modelCatBoost.fit(X_train, y_train,cat_features=categorical_features_indices,eval_set=(X_validation, y_validation),plot=True)

In [24]:
preds = modelCatBoost.predict(X_train)
logloss = log_loss(y_train, preds)
print("logloss: %f" % (logloss))

logloss: 0.361421


In [25]:
%%capture
modelCatBoost.fit(train_data,train_label,cat_features=categorical_features_indices)
preds = modelCatBoost.predict(test)
entregable_CatBoost = IDs[['Opportunity_ID']].copy()
entregable_CatBoost['Target'] = preds

In [26]:
entregable_CatBoost.to_csv('CatBoost.csv', index=False)

__Grid Search Cross Validation__

In [27]:
modeloCB = CatBoostRegressor()
LogLoss = make_scorer(log_loss, greater_is_better = False)
categorical_features_indices = np.where(train_data.dtypes == np.object)[0]

In [28]:
parameters = {'iterations':[100, 250, 500] , 'depth':[2, 3, 5], 'learning_rate':[0.005, 0.05, 0.5]}
gscv = GridSearchCV(modeloCB, parameters, cv=5, scoring=LogLoss)

In [29]:
%%capture
gscv.fit(train_data, train_label, cat_features = categorical_features_indices)

In [30]:
print(gscv.best_params_)

{'depth': 5, 'iterations': 100, 'learning_rate': 0.05}


__Predicción__

In [31]:
%%capture
modeloCB = CatBoostRegressor(depth = 5, iterations = 100, learning_rate = 0.05)
modeloCB.fit(train_data, train_label, cat_features = categorical_features_indices)
result = modeloCB.predict(test)

entregable_CB = IDs[['Opportunity_ID']].copy()
entregable_CB['Target'] = result

In [32]:
entregable_CB.to_csv('CB_Final.csv', index=False)